# Testing rescaling function
The function we are creating will be useful in order to convert the resolution of a latitude-longitude grid. 

Input parameters : 



*   da: xarray containg data to interpolate with "latitude" and "longitude" among levels of indexes
*   int : target resolution (could be grater or lower than the original)
*   method : String "linear" or "nearest"(Default)

Return :


*   xarray with new index (computed with target resolution and starting values of original indexes) and interpolated values of variables (depending the method you choose)









## Install dependencies

In [1]:
# Install library to read netcds files
!pip install netcdf4

Next cell mount in the colab notebook the google drive folder, click on the link that appears under the cell, login and copy the string into the field of the cell to connect.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Rebase the current directory to a directory called "ESoWC", make sure you created it before in your drive. 
(It should create it automatically but you never know :) )

In [3]:
%cd /content/drive/My Drive/ESoWC

/content/drive/My Drive/ESoWC


## Function: scale grid

In order ot test the function we will select a small portion of the original CAMS  concentrations measurements and analyze interpolation results on such small frame.
For easy visualization we apply the function to xarray, but visualize the pandas dataframe.

In [4]:
import xarray as xr

filename = 'Data/CAMS_analysis_MAY_2019_00_24.nc'
da = xr.open_dataset(filename)
da

<xarray.Dataset>
Dimensions:     (latitude: 15, level: 1, longitude: 15, time: 744)
Coordinates:
  * longitude   (longitude) float32 8.55 8.65 8.75 8.85 ... 9.65 9.75 9.85 9.95
  * latitude    (latitude) float32 45.95 45.85 45.75 45.65 ... 44.75 44.65 44.55
  * level       (level) float32 0.0
  * time        (time) timedelta64[ns] 00:00:00 01:00:00 ... 30 days 23:00:00
Data variables:
    co_conc     (time, level, latitude, longitude) float32 ...
    no2_conc    (time, level, latitude, longitude) float32 ...
    no_conc     (time, level, latitude, longitude) float32 ...
    o3_conc     (time, level, latitude, longitude) float32 ...
    pm10_conc   (time, level, latitude, longitude) float32 ...
    pm2p5_conc  (time, level, latitude, longitude) float32 ...
Attributes:
    title:        CO/NO2/NO/O3/PM10/PM25 Air Pollutant ANALYSIS at the Surface
    institution:  Data produced by Meteo France
    source:       Data from ENSEMBLE model
    history:      Model ENSEMBLE ANALYSIS
    ANALYSIS:     Europe, 20190501-20190531+[0H_23H]
    summary:      ENSEMBLE model hourly ANALYSIS of CO/NO2/NO/O3/PM10/PM25 co...
    project:      MACC-RAQ (http://macc-raq.gmes-atmosphere.eu)

In [5]:
sel = da.where(da.latitude <=44.65  , drop = True)
reduced = sel.where(sel.longitude <= 8.65, drop = True)
reduced

<xarray.Dataset>
Dimensions:     (latitude: 2, level: 1, longitude: 2, time: 744)
Coordinates:
  * longitude   (longitude) float32 8.55 8.65
  * latitude    (latitude) float32 44.65 44.55
  * level       (level) float32 0.0
  * time        (time) timedelta64[ns] 00:00:00 01:00:00 ... 30 days 23:00:00
Data variables:
    co_conc     (time, level, latitude, longitude) float32 166.3 174.8 ... 180.8
    no2_conc    (time, level, latitude, longitude) float32 6.611 9.064 ... 12.32
    no_conc     (time, level, latitude, longitude) float32 0.006509 ... 0.00594
    o3_conc     (time, level, latitude, longitude) float32 97.87 98.71 ... 88.09
    pm10_conc   (time, level, latitude, longitude) float32 12.87 14.36 ... 13.85
    pm2p5_conc  (time, level, latitude, longitude) float32 9.137 9.443 ... 13.37
Attributes:
    title:        CO/NO2/NO/O3/PM10/PM25 Air Pollutant ANALYSIS at the Surface
    institution:  Data produced by Meteo France
    source:       Data from ENSEMBLE model
    history:      Model ENSEMBLE ANALYSIS
    ANALYSIS:     Europe, 20190501-20190531+[0H_23H]
    summary:      ENSEMBLE model hourly ANALYSIS of CO/NO2/NO/O3/PM10/PM25 co...
    project:      MACC-RAQ (http://macc-raq.gmes-atmosphere.eu)

In [6]:
#get the pandas dataframe
df = reduced.to_dataframe()
df

co_conc  ...  pm2p5_conc
latitude  level longitude time                          ...            
44.650002 0.0   8.55      0 days 00:00:00   166.303879  ...    9.136590
                          0 days 01:00:00   163.358307  ...    9.639406
                          0 days 02:00:00   157.803543  ...    9.960825
                          0 days 03:00:00   156.223511  ...    9.901752
                          0 days 04:00:00   155.652863  ...    9.694771
...                                                ...  ...         ...
44.549999 0.0   8.65      30 days 19:00:00  153.359818  ...   13.699776
                          30 days 20:00:00  165.859299  ...   14.375214
                          30 days 21:00:00  176.851822  ...   14.789965
                          30 days 22:00:00  181.507019  ...   14.345167
                          30 days 23:00:00  180.798126  ...   13.366114

[2976 rows x 6 columns]

In [7]:
import numpy as np


#Split "time" index into "day" and "hour"
df['Days'] = df.index.get_level_values("time").days 
df['Hours'] = df.index.get_level_values("time").seconds//3600
df.set_index('Days', append=True, inplace=True)
df.set_index('Hours', append=True, inplace=True)
# round coordinates
df['lat'] = np.around(df.index.get_level_values("latitude"),2)
df['long'] = np.around(df.index.get_level_values("longitude"),2)
df.set_index('lat', append=True, inplace=True)
df.set_index('long', append=True, inplace=True)
#Drop "level" and "time" indexes and old "latitude" and "longitude" indexes
df.reset_index(level=[0,1,2,3], drop=True, inplace=True)
df = df.reorder_levels(['lat','long','Days', 'Hours'])
df = df.rename_axis(index=['latitude', 'longitude','Days', 'Hours'])

In [8]:
df.head()

co_conc  no2_conc  ...  pm10_conc  pm2p5_conc
latitude longitude Days Hours                        ...                       
44.65    8.55      0    0      166.303879  6.610581  ...  12.873794    9.136590
                        1      163.358307  5.797691  ...  15.227795    9.639406
                        2      157.803543  5.202508  ...  15.121656    9.960825
                        3      156.223511  4.489035  ...  14.549563    9.901752
                        4      155.652863  4.132488  ...  14.079832    9.694771

[5 rows x 6 columns]

In [9]:
df_selection = df.loc[df.index.get_level_values('Days') == 0 ] 
df_selection

co_conc  no2_conc  ...  pm10_conc  pm2p5_conc
latitude longitude Days Hours                        ...                       
44.65    8.55      0    0      166.303879  6.610581  ...  12.873794    9.136590
                        1      163.358307  5.797691  ...  15.227795    9.639406
                        2      157.803543  5.202508  ...  15.121656    9.960825
                        3      156.223511  4.489035  ...  14.549563    9.901752
                        4      155.652863  4.132488  ...  14.079832    9.694771
...                                   ...       ...  ...        ...         ...
44.55    8.65      0    19     163.655258  5.394433  ...  13.648726   12.442271
                        20     160.770355  7.640146  ...  13.724623   11.301576
                        21     161.823700  7.357179  ...  14.640473   10.683568
                        22     165.873016  6.643798  ...  14.403239   10.215868
                        23     167.324432  6.048254  ...  15.537005   10.359750

[96 rows x 6 columns]

In [10]:
df_select_hourly = df_selection.loc[df_selection.index.get_level_values('Hours' )<= 4]
df_select_hourly 

co_conc  no2_conc  ...  pm10_conc  pm2p5_conc
latitude longitude Days Hours                        ...                       
44.65    8.55      0    0      166.303879  6.610581  ...  12.873794    9.136590
                        1      163.358307  5.797691  ...  15.227795    9.639406
                        2      157.803543  5.202508  ...  15.121656    9.960825
                        3      156.223511  4.489035  ...  14.549563    9.901752
                        4      155.652863  4.132488  ...  14.079832    9.694771
         8.65      0    0      174.830841  9.064035  ...  14.357748    9.443238
                        1      170.862778  7.773185  ...  15.259172    9.844498
                        2      162.089539  6.700667  ...  14.444993    9.068241
                        3      158.350250  5.615870  ...  14.040246   10.070520
                        4      156.923645  5.353683  ...  13.807581    8.905330
44.55    8.55      0    0      161.228439  6.017477  ...  13.415171    8.935563
                        1      155.539841  5.553236  ...  15.926343    9.566974
                        2      152.060532  4.738773  ...  15.514559    9.653743
                        3      150.001541  4.275709  ...  14.916999    9.699616
                        4      149.475342  4.398557  ...  14.933478    9.861585
         8.65      0    0      169.549637  8.210297  ...  15.479155    9.425276
                        1      161.104584  6.938972  ...  15.932511   10.502929
                        2      153.035400  5.798157  ...  16.303034   10.367817
                        3      150.525299  5.355293  ...  15.811516    9.846500
                        4      150.135864  5.259565  ...  14.536939    8.886052

[20 rows x 6 columns]

In [11]:
da = df_select_hourly.to_xarray()
da

<xarray.Dataset>
Dimensions:     (Days: 1, Hours: 5, latitude: 2, longitude: 2)
Coordinates:
  * latitude    (latitude) float64 44.55 44.65
  * longitude   (longitude) float64 8.55 8.65
  * Days        (Days) int64 0
  * Hours       (Hours) int64 0 1 2 3 4
Data variables:
    co_conc     (latitude, longitude, Days, Hours) float32 161.2 155.5 ... 156.9
    no2_conc    (latitude, longitude, Days, Hours) float32 6.017 5.553 ... 5.354
    no_conc     (latitude, longitude, Days, Hours) float32 0.00297 ... 0.01245
    o3_conc     (latitude, longitude, Days, Hours) float32 107.1 106.8 ... 95.21
    pm10_conc   (latitude, longitude, Days, Hours) float32 13.42 15.93 ... 13.81
    pm2p5_conc  (latitude, longitude, Days, Hours) float32 8.936 9.567 ... 8.905

In [16]:
import numpy as np
import math 
#from rescaling import rescale_grid
def rescale_grid(data, target_res, method = 'nearest'):
  
  assert ('latitude' in data.variables),"latitude column missing (name must be 'latitude')"
  assert ('longitude' in data.variables),"longitude column missing (name must be 'longitude')"

  #da = data.to_array()

  lats = data.latitude.values
  longs = data.longitude.values

  lats.sort()
  longs.sort()

  #lats_res = round(abs(lats[1] - lats[0]), 2) # supposing at least 2 values and max_precision = 2 decimals and res_lat = res_long
  #lat_interval = abs(round(lats[-1],2) - round(lats[0], 2))
  #long_interval = abs(round(longs[-1],2) - round(longs[0], 2))
  lat_interval = np.float32(lats[-1] - lats[0])
  long_interval = np.float32(longs[-1] - longs[0])
  #print("lat interval is ",lat_interval)
  #print("long interval is --> " , long_interval)
  lat_new_squares = lat_interval // target_res
  #print(lat_new_squares)
  long_new_squares = long_interval // target_res
  #print(long_new_squares)
  new_lat_values= np.around(np.arange(0, lat_new_squares +1 , 1) * target_res + round(lats[0], 2), decimals=2)
  #print("New latitude values are -> ", new_lat_values)
  new_long_values= np.around(np.arange(0, long_new_squares +1 , 1) * target_res + round(longs[0], 2), decimals=2)
  #print("New longitude values are -> ", new_long_values)
  #da = da.sortby(['latitude','longitude','time'])
  #df_temp = data.interp(latitude = new_lat_values, longitude = new_long_values, method = method)
  #df= df_temp.interp(longitude = new_long_values, method = method)

  return data.interp(latitude = new_lat_values, longitude = new_long_values, method = method)

da_rescale =rescale_grid(da, 0.05, 'linear')

In [17]:
#df_rescale = da_final.to_dataframe(name = 'df_scaled')
#df_rescale = da_final.to_dataframe()
df = da_rescale.to_dataframe()
df = df.reorder_levels(['latitude','longitude','Days', 'Hours'])

df

co_conc  no2_conc  ...  pm10_conc  pm2p5_conc
latitude longitude Days Hours                        ...                       
44.55    8.55      0    0      161.228439  6.017477  ...  13.415171    8.935563
         8.60      0    0      165.389038  7.113887  ...  14.447163    9.180419
         8.65      0    0      169.549637  8.210297  ...  15.479155    9.425276
44.60    8.55      0    0      163.766159  6.314029  ...  13.144482    9.036077
         8.60      0    0      167.978199  7.475597  ...  14.031467    9.235167
         8.65      0    0      172.190239  8.637166  ...  14.918451    9.434257
44.65    8.55      0    0      166.303879  6.610581  ...  12.873794    9.136590
         8.60      0    0      170.567360  7.837308  ...  13.615771    9.289914
         8.65      0    0      174.830841  9.064035  ...  14.357748    9.443238
44.55    8.55      0    1      155.539841  5.553236  ...  15.926343    9.566974
         8.60      0    1      158.322212  6.246104  ...  15.929427   10.034951
         8.65      0    1      161.104584  6.938972  ...  15.932511   10.502929
44.60    8.55      0    1      159.449074  5.675464  ...  15.577069    9.603190
         8.60      0    1      162.716377  6.515771  ...  15.586455    9.888452
         8.65      0    1      165.983681  7.356079  ...  15.595842   10.173713
44.65    8.55      0    1      163.358307  5.797691  ...  15.227795    9.639406
         8.60      0    1      167.110542  6.785438  ...  15.243484    9.741952
         8.65      0    1      170.862778  7.773185  ...  15.259172    9.844498
44.55    8.55      0    2      152.060532  4.738773  ...  15.514559    9.653743
         8.60      0    2      152.547966  5.268465  ...  15.908796   10.010780
         8.65      0    2      153.035400  5.798157  ...  16.303034   10.367817
44.60    8.55      0    2      154.932037  4.970641  ...  15.318108    9.807284
         8.60      0    2      156.247253  5.610026  ...  15.346061    9.762656
         8.65      0    2      157.562469  6.249412  ...  15.374013    9.718029
44.65    8.55      0    2      157.803543  5.202508  ...  15.121656    9.960825
         8.60      0    2      159.946541  5.951588  ...  14.783325    9.514533
         8.65      0    2      162.089539  6.700667  ...  14.444993    9.068241
44.55    8.55      0    3      150.001541  4.275709  ...  14.916999    9.699616
         8.60      0    3      150.263420  4.815501  ...  15.364257    9.773058
         8.65      0    3      150.525299  5.355293  ...  15.811516    9.846500
44.60    8.55      0    3      153.112526  4.382372  ...  14.733281    9.800684
         8.60      0    3      153.775150  4.933977  ...  14.829581    9.879597
         8.65      0    3      154.437775  5.485582  ...  14.925881    9.958510
44.65    8.55      0    3      156.223511  4.489035  ...  14.549563    9.901752
         8.60      0    3      157.286880  5.052453  ...  14.294905    9.986136
         8.65      0    3      158.350250  5.615870  ...  14.040246   10.070520
44.55    8.55      0    4      149.475342  4.398557  ...  14.933478    9.861585
         8.60      0    4      149.805603  4.829061  ...  14.735209    9.373818
         8.65      0    4      150.135864  5.259565  ...  14.536939    8.886052
44.60    8.55      0    4      152.564102  4.265522  ...  14.506655    9.778178
         8.60      0    4      153.046928  4.786073  ...  14.339458    9.336934
         8.65      0    4      153.529755  5.306624  ...  14.172260    8.895691
44.65    8.55      0    4      155.652863  4.132488  ...  14.079832    9.694771
         8.60      0    4      156.288254  4.743085  ...  13.943707    9.300050
         8.65      0    4      156.923645  5.353683  ...  13.807581    8.905330

[45 rows x 6 columns]